<a href="https://colab.research.google.com/github/MostakemHossain/8-pazzle-problem-solve-using-A-/blob/main/Ai_final_project_showcase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import heapq, copy, time
from matplotlib import animation
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

: 

In [ ]:
# Puzzle state representation
class PuzzleState:
    def __init__(self, state, parent=None, action=None, cost=0):
        self.state = state  # 2D numpy array representing the puzzle
        self.parent = parent  # Reference to parent state
        self.action = action  # Move taken to reach this state
        self.cost = cost  # Cost (number of moves from the initial state)
        self.dimension = len(state)  # Dimension of puzzle (3x3)
        self.empty_pos = self.find_empty_position()  # Position of the empty tile (0)

    # Find the (i, j) of the empty tile
    def find_empty_position(self):
        for i in range(self.dimension):
            for j in range(self.dimension):
                if self.state[i][j] == 0:
                    return (i, j)
        return None

    def __lt__(self, other):
        return self.cost < other.cost  # Required for priority queue

    def __eq__(self, other):
        return np.array_equal(self.state, other.state)

    def __hash__(self):
        return hash(str(self.state))

# Manhattan distance heuristic function
def get_manhattan_distance(state, goal_state):
    distance = 0
    for i in range(state.dimension):
        for j in range(state.dimension):
            val = state.state[i][j]
            if val != 0:
                gx, gy = np.where(goal_state.state == val)
                distance += abs(gx[0] - i) + abs(gy[0] - j)
    return distance

In [ ]:
# Return list of possible actions from current state
def get_possible_actions(state):
    i, j = state.empty_pos
    actions = []
    if i > 0: actions.append('up')
    if i < 2: actions.append('down')
    if j > 0: actions.append('left')
    if j < 2: actions.append('right')
    return actions

In [ ]:
# Generate new state by applying an action
def get_new_state(state, action):
    i, j = state.empty_pos
    new_state = copy.deepcopy(state.state)
    if action == 'up': new_state[i][j], new_state[i-1][j] = new_state[i-1][j], new_state[i][j]
    elif action == 'down': new_state[i][j], new_state[i+1][j] = new_state[i+1][j], new_state[i][j]
    elif action == 'left': new_state[i][j], new_state[i][j-1] = new_state[i][j-1], new_state[i][j]
    elif action == 'right': new_state[i][j], new_state[i][j+1] = new_state[i][j+1], new_state[i][j]
    return PuzzleState(new_state, state, action, state.cost + 1)

In [ ]:
# Check if a puzzle is solvable using inversion count
def is_solvable(state):
    flat = [num for row in state for num in row if num != 0]
    inv = sum(1 for i in range(len(flat)) for j in range(i + 1, len(flat)) if flat[i] > flat[j])
    return inv % 2 == 0

In [ ]:
# A* search algorithm
def a_star_search(initial_state, goal_state):
    frontier = []
    explored = set()
    initial_cost = get_manhattan_distance(initial_state, goal_state)
    heapq.heappush(frontier, (initial_cost, initial_state))

    while frontier:
        _, current = heapq.heappop(frontier)
        if np.array_equal(current.state, goal_state.state):
            return current
        explored.add(hash(str(current.state)))

        for action in get_possible_actions(current):
            new_state = get_new_state(current, action)
            if hash(str(new_state.state)) not in explored:
                f_cost = new_state.cost + get_manhattan_distance(new_state, goal_state)
                heapq.heappush(frontier, (f_cost, new_state))
    return None

In [ ]:
# Reconstruct path from goal to initial state
def get_solution_path(solution):
    path = []
    while solution:
        path.append(solution)
        solution = solution.parent
    return path[::-1]

In [ ]:
# Generate a random solvable puzzle state
def create_random_state():
    while True:
        flat = np.random.permutation(9)
        matrix = flat.reshape(3, 3)
        if is_solvable(matrix):
            return PuzzleState(matrix)

**UI & Visualization**

In [ ]:
# Draw a single state on the given axis
def visualize_state(state, ax):
    cmap = ListedColormap(['white', 'lightblue'])
    mask = (state.state == 0)
    ax.imshow(~mask, cmap=cmap, interpolation='nearest')
    for i in range(3):
        for j in range(3):
            if state.state[i][j] != 0:
                ax.text(j, i, str(state.state[i][j]), ha='center', va='center', fontsize=20, fontweight='bold')
    ax.set_xticks([]); ax.set_yticks([]); ax.grid(False)

In [ ]:
# Animate the sequence of puzzle steps
def animate_solution(path, steps_to_show=1):
    fig, ax = plt.subplots(figsize=(4, 4))

    def update(frame):
        ax.clear()
        visualize_state(path[frame], ax)
        ax.set_title(f"Step {frame + 1}/{len(path)}")

    total_frames = len(path)
    frames_to_animate = list(range(0, total_frames, steps_to_show))

    anim = animation.FuncAnimation(fig, update, frames=frames_to_animate, interval=700, repeat=False)
    plt.close(fig)
    return anim

In [ ]:
# Show each puzzle step in a row-wise layout
def display_steps_row_wise(path, max_per_row=4):
    num_steps = len(path)
    rows = (num_steps + max_per_row - 1)

    fig, axes = plt.subplots(rows, max_per_row, figsize=(5 * max_per_row, 5 * rows))

    if rows == 1:
        axes = [axes]  # Ensure it's a list for uniformity
    else:
        axes = axes.reshape(rows, max_per_row)

    for idx, state in enumerate(path):
        row, col = divmod(idx, max_per_row)
        ax = axes[row][col]
        visualize_state(state, ax)
        ax.set_title(f"Step {idx + 1}", fontsize=12)

    for idx in range(num_steps, rows * max_per_row):
        row, col = divmod(idx, max_per_row)
        axes[row][col].axis("off")

    plt.tight_layout()
    plt.show()


In [ ]:

# Create the full UI layout and interactivity
def create_puzzle_ui():
    initial_state = create_random_state()
    goal_state = create_random_state()

    # Output widgets for visualization and messages
    output_init = widgets.Output()
    output_goal = widgets.Output()
    output_solution = widgets.Output()
    output_steps = widgets.Output()
    output_status = widgets.Output()

    # Input and control widgets
    txt_input = widgets.Text(description='Custom Input:', placeholder='e.g., 1,2,3,4,5,6,7,8,0')
    btn_solve = widgets.Button(description="Solve", button_style='success')
    btn_new = widgets.Button(description="New Puzzle", button_style='info')
    btn_random_goal = widgets.Button(description="Randomize Goal", button_style='warning')

    # Draw the current puzzle and goal states
    def draw_states():
        with output_init:
            clear_output()
            plt.figure(figsize=(3, 3))
            visualize_state(initial_state, plt.gca())
            plt.title("Initial State")
            plt.show()
        with output_goal:
            clear_output()
            plt.figure(figsize=(3, 3))
            visualize_state(goal_state, plt.gca())
            plt.title("Goal State")
            plt.show()

    # When "New Puzzle" button is clicked
    def on_new_clicked(_):
        nonlocal initial_state
        initial_state = create_random_state()
        draw_states()
        output_solution.clear_output()
        output_steps.clear_output()
        output_status.clear_output()

    # When "Randomize Goal" button is clicked
    def on_random_goal_clicked(_):
        nonlocal goal_state
        goal_state = create_random_state()
        draw_states()
        output_solution.clear_output()
        output_steps.clear_output()
        output_status.clear_output()

    # Handle custom puzzle input from user
    def on_custom_input_entered(change):
        nonlocal initial_state
        try:
            nums = list(map(int, change['new'].split(',')))
            if sorted(nums) != list(range(9)):
                raise ValueError
            matrix = np.array(nums).reshape(3, 3)
            if not is_solvable(matrix):
                raise ValueError("Puzzle is unsolvable")
            initial_state = PuzzleState(matrix)
            draw_states()
        except Exception:
            with output_status:
                clear_output()
                print("❌ Invalid input! Enter 9 unique numbers from 0 to 8 in CSV format.")

    # When "Solve" button is clicked
    def on_solve_clicked(_):
        with output_status:
            clear_output()
            print("⏳ Solving... Please wait.")
        with output_solution:
            clear_output()
        with output_steps:
            clear_output()

        start = time.time()
        solution = a_star_search(initial_state, goal_state)
        duration = time.time() - start

        with output_solution:
            if solution:
                path = get_solution_path(solution)
                anim = animate_solution(path, steps_to_show=2)
                display(anim)
                with output_status:
                    clear_output()
                    print(f"✅ Solved in {len(path) - 1} moves, Time: {duration:.2f} sec")
                with output_steps:
                    display_steps_row_wise(path)
            else:
                with output_status:
                    clear_output()
                    print("❌ No solution found!")

    # Attach button click events
    btn_new.on_click(on_new_clicked)
    btn_random_goal.on_click(on_random_goal_clicked)
    btn_solve.on_click(on_solve_clicked)
    txt_input.observe(on_custom_input_entered, names='value')

    draw_states()

    # Layout structure
    layout = widgets.VBox([
        widgets.HTML("<h2>🧩 8-Puzzle Solver with A* Search</h2>"),
        widgets.HBox([
            widgets.VBox([widgets.HTML("<b>Initial State</b>"), output_init]),
            widgets.VBox([widgets.HTML("<b>Goal State</b>"), output_goal])
        ]),
        txt_input,
        widgets.HBox([btn_solve, btn_new, btn_random_goal]),
        output_status,
        output_solution,
        output_steps
    ])

    return layout


Run UI

In [ ]:
ui = create_puzzle_ui()
display(ui)
